In [1]:
import pandas, numpy

N_ROWS = 3


# II. Normalizing Addresses:
## Abstract 
Some address are very similar and ought to be grouped together. 

Using fuzzy matching 42,368 addresses were matched to 15,416 unique addresses from a total of 93,454.

Problems were found (4,154, or  26% of the resulting parent addresses), creating trees of relationships.
NOTE: not exact :
The address database is hereby reduced by 45% with 51,086 remaining unique addresses.

## 0. Import and exploration:
Loading relevant columns, all strings to lowercase, removing punctuation, and creating an empty parent_id column.

In [2]:
address = pandas.read_csv('data/nodes.address.csv', usecols=['node_id', 'address', 'countries'], index_col='node_id')
address = address.applymap( lambda _str: _str.lower() if isinstance( _str,str ) else _str )

def remove_punctuation(_str):
    for punc in [ ';', '#', ',', '|', '\t' ]:
        _str = _str.replace( punc, ' ' )
    return _str
    
address.address = address.address.map( lambda _str: remove_punctuation(_str) if isinstance( _str,str) else _str )

address.head(N_ROWS)

,address,countries
node_id,,
14000001,- 27 rosewood drive 16-19 singapore 737920,singapore
14000002,"""almaly village"" v.5 almaty kazakhstan",kazakhstan
14000003,"""cantonia"" south road st georges hill weybridg...",united kingdom


Very few address are direct duplicates:

In [3]:
def get_duplicates( df, col ):
    duplicates = df[ df.duplicated([col]) ]

    total = df.shape[0]
    unique = len( df[col].unique() )

    duplicate_count = total - unique
    duplicate_percent = int( (duplicate_count * 100) / total )

    print( f'Total entries: {total} \nDuplicates: {duplicate_count} or {duplicate_percent}%' )
    return duplicates

duplicates = get_duplicates( address, 'address' )
print(f'Duplicates: {duplicates}')

Total entries: 93454 
Duplicates: 6 or 0%
Duplicates:                                                     address   countries
node_id                                                                
14021332       6 salah el din street  zamalek  cairo  egypt       egypt
14022103                  77 robinson road  31-00 singapore   singapore
14028059  arango orillac bldg.  2nd floor  54th street  ...      panama
14065612           rincon 531  piso 7  oficina 702  uruguay     uruguay
14077667  second floor  forum 4  grenville street  st he...      jersey
14079083  suite 13  first floor  oliaji trade centre  fr...  seychelles


## 1. Sanitizing street address:

Removing the country from the address strings:

The more edge cases we remove the better results we get from computing similarities.

TODO: Extract and remove cities for better results; { '60 piers road; borrowdale; harare': ('12 hunt road borrowdale, harare', 79), ('12 hunt road; borrrowdale; harare', 77) }

In [4]:
def remove_country(text, *country_list):
    if type(text) == float:
        return text 
    for country in country_list:
        if country in text:
            text = text.replace(country, '')
    return text.strip()

country_list = list( address.countries.dropna().unique() )

### add edge cases ###
st_vincent = ['st. vincent & grenadines', 'st vincent and the grenadines', 'st vincent & the grenadines', 
            'saint-vincent and grenadines', 'saint-vincent', 'st. vincent', 'saint vincent', '& the grenadines']
country_list = st_vincent + country_list
country_list += ['vietnam', 'sengal', 'europe', 'republic of', 'zimbawe', 'zimabwe']

address.address = address['address'].apply(remove_country, args=country_list)
address.head(N_ROWS)

,address,countries
node_id,,
14000001,- 27 rosewood drive 16-19 737920,singapore
14000002,"""almaly village"" v.5 almaty",kazakhstan
14000003,"""cantonia"" south road st georges hill weybridg...",united kingdom


## 2. Fuzzy matching:

#### !! Computationly heavy !!
It is recomended to run this script on anything else than a Thinkpad X270 (Elapsed time: 1h30min), Amazon AWS being one of the better options. This notebook is thus deliberately limited to groups containing less than 50 unique strings. The actual results are included and loaded later.

TODO: Find Big-O Notation

In [5]:
from multiprocessing import Pool, cpu_count
from fuzzywuzzy import fuzz, process
import time

# TODO: in production; yield the results as they come to the rest of the pipeline.
# TODO: sort the groups by size, so the computation can start on the biggest group first (i.e: China with 20k address). This would decrease the total computation time.

THRESHOLD = 75
SELECTOR = 'address'
GROUP = address.groupby('countries')
LIMIT = 50

def calculate_string_similarity(df):
    t1 = time.time()

    choices = set( df[ SELECTOR ].unique() )
    seen = set()
    results = dict()

    choice_count = len(choices)
    for i in range( choice_count-1 ):
        choice = list(choices)[i]
        if len(choice) < 2:
            i += 1
            continue

        seen.add(choice)
        new_choices = choices.difference(seen)
        if len(new_choices) <= 1:
            break

        res = process.extract(choice, new_choices, scorer=fuzz.token_sort_ratio, limit=10000)
        res = [ r[0] for r in res if r[1] > THRESHOLD ]

        if len(res):
            seen.update(res)
            results[choice] = res
        i += 1 

    tt = time.time() - t1
    print( f'Compared {choice_count} in {tt} sec' )
    return results

def gapply_parallel(df_group, func):
    t1 = time.time()

    if LIMIT:
        df_list = [ group for name, group in df_group if group.shape[0] < LIMIT ]
    else:
        df_list = [ group for name, group in df_group ]

    results = {}
    with Pool( cpu_count() ) as pool:
        for res in pool.map(func, df_list):
            results.update( res )

    results = pandas.DataFrame.from_dict(results,  orient='index')
    results.reset_index(inplace=True)
    #results = results.rename( {'index':'parent'} , axis=1)

    print( f'Total in:', time.time()-t1 )
    return results


similar_str = gapply_parallel(GROUP, calculate_string_similarity)
similar_str = similar_str.rename( {'index':'parent'} , axis=1)

Compared 1 in 0.0018134117126464844 sec
Compared 18 in 0.008359670639038086 sec
Compared 23 in 0.020578622817993164 sec
Compared 17 in 0.010998725891113281 sec
Compared 17 in 0.00950932502746582 sec
Compared 19 in 0.034926652908325195 sec
Compared 10 in 0.003962993621826172 sec
Compared 38 in 0.05935072898864746 sec
Compared 22 in 0.042871713638305664 sec
Compared 35 in 0.04048776626586914 sec
Compared 12 in 0.005051136016845703 sec
Compared 38 in 0.03825998306274414 sec
Compared 19 in 0.019723892211914062 sec
Compared 45 in 0.10547828674316406 sec
Compared 18 in 0.014898300170898438 sec
Compared 7 in 0.0019183158874511719 sec
Compared 23 in 0.01829242706298828 sec
Compared 22 in 0.03971695899963379 sec
Compared 38 in 0.08525586128234863 sec
Compared 35 in 0.037343740463256836 sec
Compared 3 in 0.0007693767547607422 sec
Compared 2 in 0.0005311965942382812 sec
Compared 3 in 0.0008034706115722656 sec
Compared 31 in 0.02710413932800293 sec
Compared 4 in 0.0008406639099121094 sec
Compared 

In [6]:
similar_str[ 15:18 ]

,parent,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
15,6 temple street st. john’s antigua w.i.,6 temple street st. john's antigua w.i.),temple street st. john's antigua w.i.,^ temple street st. john's antigua w.i.,of 6 temple street st. john's antigua w.i.,6 temple street st. john's antigua,6 temple street st. john's antigua west indies,6 temple stret st john's,None,None,None,None,None,None,None,None,None,None,None
16,23-22 hrachya kochar street yerevan,23-29 hrachya kochar street yerevan,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
17,15-3 sundukyan st. erevan,15-3 sundukyana st. erevan,sundukyan str. 15-3 erevan .,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## 3. Creating addresss edges:
Map all address strings to their node_id, using all cores available. 

#### Big-O = O*n

In [7]:
def string_to_ID(df):
    def func(x):
        if isinstance(x, str):
            return address.index[ address.address == x ][0]
        return 0
        
    return df.applymap(func)

def apply_parallel(df, func):
    t1 = time.time()

    _cpu_count = cpu_count()
    df_split = numpy.array_split( df, _cpu_count )

    with Pool( _cpu_count ) as pool:
        res = pool.map( func, df_split )
        try:
            df = pandas.concat( res )
        except ValueError:
            # result could be a list of Nones
            pass 
            
    print(f'Total time:', time.time()-t1 )
    return df

similar_nodes = apply_parallel(similar_str, string_to_ID)
similar_nodes.tail(N_ROWS)

Total time: 4.373776435852051


,parent,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
176,14000827,14089215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
177,14023143,14023144,14062647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
178,14062647,14062648,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


As parent/child pair:

In [8]:
def flatten_to_pair(df):
    return  ( similar_nodes.astype('Int64').where(similar_nodes.ne(0))
                    .set_index('parent')
                    .stack()
                    .reset_index(level=0, name='child_id') )

similar_nodes = apply_parallel(similar_nodes, flatten_to_pair)
similar_nodes.rename( { 'parent' : 'parent_id'}, axis=1, inplace=True)
similar_nodes.reset_index(drop=True, inplace=True)

similar_nodes.head(N_ROWS)

Total time: 0.4342162609100342


,parent_id,child_id
0,14103793,14103880
1,14062504,14026529
2,14036094,14025454


## 4. Resolving problems to better quantify the results:
Importing the pre-calculated full results from "data/edges.address.csv" and quantifying the results. Problems of type "master->parent->child" trees are counted.

In [9]:
similar_nodes = pandas.read_csv('data/edges.address.csv')

def quantify_results():
    original_count = address.shape[0]

    parents = set( similar_nodes.parent_id )
    unique_parents = len( parents )

    children = set( similar_nodes.child_id )
    unique_children = len( children )

    problems = list( parents.intersection( children ))
    problem_count = len( problems )
    problem_percent = int( (problem_count * 100) / unique_parents )

    remaining = original_count - unique_children + problem_count
    reduction = int( 100 - (( remaining * 100 ) / original_count ))

    print( f"""* Reduced {unique_children} to {unique_parents} from a total of {original_count}.
            \n* Problems found: {problem_count} or {problem_percent}% of the parents.
            \n* Remaining: {remaining}. Reduction: -{reduction}%""" )
    return problems

problems = quantify_results()

* Reduced 42368 to 15416 from a total of 93454.
            
* Problems found: 4154 or 26% of the parents.
            
* Remaining: 55240. Reduction: -40%


The next bit of code enables you to trace these problems. 

In [10]:
def trace_problem(prob):
    as_parent = similar_nodes[ similar_nodes.parent_id == prob ]
    as_child = similar_nodes[ similar_nodes.child_id == prob ]

    print(f'As parent: \n{as_parent}\nAs child:\n{as_child}')

    masters = list( as_child.parent_id )
    for i in range( len(masters) ): 
        master = similar_nodes[ similar_nodes.parent_id == masters[i] ]
        i += 1
        print(f'Master {i}:\n{master}')

    print('\nTrace address:')
    print( list( address[ address.index == prob ]['address'] ))
    print('\nMaster address:')
    print( list( address[ address.index == as_child.parent_id.iloc[0]]['address'] ))

trace_problem(problems[5])

As parent: 
       parent_id  child_id
29924   14032904  14032901
As child:
       parent_id  child_id
29915   14048554  14032904
Master 1:
       parent_id  child_id
29912   14048554  14048558
29913   14048554  14048547
29914   14048554  14048546
29915   14048554  14032904
29916   14048554  14032903

Trace address:
['c/o jackson russell  level 3  fonterra centre  9 princes st  auckland']

Master address:
['level 3  fonterra centre  9 princes street  auckland']


The children's ownership could be transfered to the masters as follow; 

Unfortunately, this operation has to be single threaded as to not modify both the ownership of the masters and parents at the same time.

In [15]:
t1 = time.time()

for prob in problems:

    as_parent = similar_nodes[ similar_nodes.parent_id == prob ]
    as_child = similar_nodes[ similar_nodes.child_id == prob ]
    master_id = list( as_child['parent_id' ])[0]
    
    similar_nodes.loc[similar_nodes.parent_id == prob, 'parent_id'] = master_id   
tt = time.time() - t1

print( f'Total in: {tt}')

problems = quantify_results()

Total in: 0.0006613731384277344
* Reduced 42368 to 11262 from a total of 93454.
            
* Problems found: 0 or 0% of the parents.
            
* Remaining: 51086. Reduction: -45%


By grouping "master->parent->child" trees, we achive a 45% reduction of the address dataset

## 5. Saving the results;
### The results are kept as is with the trees, after all we are creating connections of similarities, not exactitudes.
The results are concatenated to "data/similarity.edges.csv".

In [17]:
# TODO: redo the whole of this

In [12]:
edges = pandas.read_csv('data/clean.edges.csv', index_col=0)
edges.start_id.isin( similar_nodes.child_id ).any()

True

In [13]:
def map_parent_id(df):
    def func(_id):
        return similar_nodes.loc[ similar_nodes.child_id == _id, 'parent_id' ].values[0]

    df.start_id = df.start_id.map( func )
    return df

matching_address_edges = edges[ edges.start_id.isin( similar_nodes.child_id ) ]
matching_address_edges = apply_parallel( matching_address_edges, map_parent_id )

edges.update( matching_address_edges )

Total time: 50.74488615989685


In [14]:
ed = edges[ edges.type == 'address' ]
len( ed.start_id.unique() )

51079